In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from tqdm.std import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from functools import partial
import sklearn.model_selection
import sklearn.preprocessing
from sklearn.metrics import recall_score

from catboost import CatBoostClassifier

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fraud_dataset.csv')
df.head()

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Grade,Sub Grade,Employment Duration,Home Ownership,Verification Status,Debit to Income,Delinquency - two years,Inquires - six months,Open Account,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Initial List Status,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status,purpose
0,10000,32236,12329.36286,59,11.135007,B,C4,MORTGAGE,176346.62670,Not Verified,16.284758,1,0,13,0,24246,74.932551,7,w,2929.646315,0.102055,2.498291,0.793724,0,49,31,311301,6619,0,debt consolidation
1,3609,11940,12191.99692,59,12.237563,C,D3,RENT,39833.92100,Source Verified,15.412409,0,0,12,0,812,78.297186,13,f,772.769385,0.036181,2.377215,0.974821,0,109,53,182610,20885,0,debt consolidation
2,28276,9311,21603.22455,59,12.545884,F,D4,MORTGAGE,91506.69105,Source Verified,28.137619,0,0,14,0,1843,2.073040,20,w,863.324396,18.778660,4.316277,1.020075,0,66,34,89801,26155,0,debt consolidation
3,11170,6954,17877.15585,59,16.731201,C,C3,MORTGAGE,108286.57590,Source Verified,18.043730,1,0,7,0,13819,67.467951,12,w,288.173196,0.044131,0.107020,0.749971,0,39,40,9189,60214,0,debt consolidation
4,16890,13226,13539.92667,59,15.008300,C,D4,MORTGAGE,44234.82545,Source Verified,17.209886,1,3,13,1,1544,85.250761,22,w,129.239553,19.306646,1294.818751,0.368953,0,18,430,126029,22579,0,refinance


In [5]:
df = df.rename(columns={'Loan Status': 'target'})

In [6]:
y = df[['target']]
X = df.drop(columns=['target'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [17]:
X_train_str = X_train.select_dtypes(include=['object'])
X_train_int = X_train.select_dtypes(include=['int64'])
X_train_float = X_train.select_dtypes(include=['float64'])
X_train_float_arr = X_train_float.to_numpy()

X_test_str = X_test.select_dtypes(include=['object'])
X_test_int = X_test.select_dtypes(include=['int64'])
X_test_float = X_test.select_dtypes(include=['float64'])
X_test_float_arr = X_test_float.to_numpy()

ohe = OneHotEncoder(sparse_output=False)
X_train_str_arr = ohe.fit_transform(X_train_str)
X_test_str_arr = ohe.transform(X_test_str)

ss = StandardScaler()
X_train_int_arr = ss.fit_transform(X_train_int)
X_test_int_arr = ss.transform(X_test_int)

X_train_arr = np.hstack([X_train_str_arr, X_train_int_arr, X_train_float_arr])
X_test_arr = np.hstack([X_test_str_arr, X_test_int_arr, X_test_float_arr])

In [20]:
X_train_new = pd.DataFrame(X_train_arr, columns=range(0, X_train_arr.shape[1]))
X_test_new = pd.DataFrame(X_test_arr, columns=range(0, X_test_arr.shape[1]))

In [72]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.001,
                           depth=10, class_weights=[0.1, 0.9])

In [73]:
model.fit(X_train_new, y_train)

0:	learn: 0.6931361	total: 92ms	remaining: 9.1s
1:	learn: 0.6930802	total: 188ms	remaining: 9.21s
2:	learn: 0.6930278	total: 283ms	remaining: 9.15s
3:	learn: 0.6929654	total: 391ms	remaining: 9.38s
4:	learn: 0.6929098	total: 490ms	remaining: 9.32s
5:	learn: 0.6928342	total: 585ms	remaining: 9.17s
6:	learn: 0.6927769	total: 679ms	remaining: 9.02s
7:	learn: 0.6927157	total: 776ms	remaining: 8.93s
8:	learn: 0.6926624	total: 868ms	remaining: 8.78s
9:	learn: 0.6926117	total: 964ms	remaining: 8.67s
10:	learn: 0.6925514	total: 1.06s	remaining: 8.54s
11:	learn: 0.6925031	total: 1.15s	remaining: 8.43s
12:	learn: 0.6924485	total: 1.24s	remaining: 8.32s
13:	learn: 0.6923863	total: 1.35s	remaining: 8.3s
14:	learn: 0.6923441	total: 1.45s	remaining: 8.2s
15:	learn: 0.6922833	total: 1.54s	remaining: 8.1s
16:	learn: 0.6922309	total: 1.63s	remaining: 7.98s
17:	learn: 0.6921716	total: 1.74s	remaining: 7.92s
18:	learn: 0.6921041	total: 1.83s	remaining: 7.81s
19:	learn: 0.6920401	total: 1.93s	remaining: 7

In [74]:
preds_class = model.predict(X_test_new)
preds_proba = model.predict_proba(X_test_new)

In [76]:
score = sklearn.metrics.accuracy_score(y_test, preds_class)
recall = sklearn.metrics.recall_score(y_test, preds_class)

In [77]:
recall

0.14321192052980133

In [78]:
sklearn.metrics.confusion_matrix(y_test, preds_class)

array([[10920,  1340],
       [ 1035,   173]])